#Mounting Drive

In [6]:
import pandas as pd
pd.options.display.max_seq_items = 200000
pd.options.display.max_colwidth = 100

In [7]:
#mounting this to my drive to access the archive.zip zip file from Kaggle that has the NYTimes dataset

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **NYTIMES dataset**



##Loading the data

In [8]:
## Unzip the NYTimes comments files

import zipfile

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/archive.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [15]:
## Read all article + comments files, and combine them in one dataframe for each with pandas
import glob
import pandas as pd

df_allcomments = pd.concat([pd.read_csv(f) for f in glob.glob('/content/Comments*.csv')])

df_allarticles = pd.concat([pd.read_csv(f) for f in glob.glob('/content/Articles*.csv')])

df_allcomments[:2]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (14,15,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (14,15,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (32,33) have mixed types.Specify d

,approveDate,articleID,articleWordCount,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,inReplyTo,newDesk,parentID,parentUserDisplayName,permID,picURL,printPage,recommendations,recommendedFlag,replyCount,reportAbuseFlag,sectionName,sharing,status,timespeople,trusted,updateDate,userDisplayName,userID,userLocation,userTitle,userURL,typeOfMaterial
0,1483455908,58691a5795d0e039260788b9,1324.0,For all you Americans out there --- still rejoicing over the majority win of Republicans over t...,20969730.0,20969730.0,<br/>,comment,1.483426e+09,1.0,0,0.0,National,0.0,NaN,20969730,https://graphics8.nytimes.com/images/apps/timespeople/none.png,1.0,5.0,NaN,0.0,NaN,Politics,0,approved,1.0,0.0,1483455908,N. Smith,64679318.0,New York City,NaN,NaN,News
1,1483455656,58691a5795d0e039260788b9,1324.0,Obamas policies may prove to be the least of this countrys worries. The GOP has been emboldened...,20969325.0,20969325.0,<br/>,comment,1.483417e+09,1.0,0,0.0,National,0.0,NaN,20969325,https://graphics8.nytimes.com/images/apps/timespeople/none.png,1.0,3.0,NaN,0.0,NaN,Politics,0,approved,1.0,0.0,1483455656,Kilocharlie,69254188.0,Phoenix,NaN,NaN,News


In [9]:
## Open one test file (feb 2018) file with pandas
import pandas as pd

feb2018article = pd.read_csv("/content/ArticlesFeb2018.csv")
feb2018comments = pd.read_csv("/content/CommentsFeb2018.csv")

feb2018article[:1]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,articleID,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,articleWordCount
0,5a7258e410f40f00018bed7d,By NICHOLAS KRISTOF,article,I Stand With the ‘She-Devils’,"['Women and Girls', ""Women's Rights"", 'United States Politics and Government', '#MeToo Movement'...",68,OpEd,23,2018-02-01 00:01:36,Unknown,"Progress on women’s rights was stalling, and then Donald Trump came along.",The New York Times,Op-Ed,https://www.nytimes.com/2018/01/31/opinion/courtland-sykes-senate-me-too.html,835


In [10]:
## Open one test file (feb 2017) file with pandas
import pandas as pd

feb2017article = pd.read_csv("/content/ArticlesFeb2017.csv")
feb2017comments = pd.read_csv("/content/CommentsFeb2017.csv")

feb2017article[:1]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,articleID,abstract,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,articleWordCount
0,58927e0495d0e0392607e1b3,NaN,By KEN BELSON,article,N.F.L. vs. Politics Has Been Battle All Season Long,"['Football', 'Super Bowl', 'National Football League', 'New England Patriots', 'Goodell, Roger',...",1,Sports,12,2017-02-02 00:26:16,Pro Football,"Despite the national tumult over immigration spilling over, Commissioner Roger Goodell said, “I’...",The New York Times,News,https://www.nytimes.com/2017/02/01/sports/super-bowl-politics-trump-nfl.html,1129


## Preprocessing the dataset

In [11]:
##Right now, each comment connected to one articleID, with three values per comment in a list: 
##[[comment_text], [is it recommended by editors (TRUE/FALSE)?], [How many likes for comment?] ]]]

def connect_comments(comments, articles, extra_comment_column=None):

  if extra_comment_column !=  None:
    comments['comment_info'] = comments[['commentBody', 'editorsSelection', 'recommendations', 'commentID', 'articleID', extra_comment_column]].values.tolist() 
  else: 
    comments['comment_info'] = comments[['commentBody', 'editorsSelection', 'recommendations', 'commentID', 'articleID']].values.tolist() 

  groupings = comments.groupby('articleID', as_index=False)

  df2 = pd.DataFrame([comments.loc[groupings.groups[n], 'comment_info'].values for n in groupings.groups], 
                    index=groupings.groups.keys())
  
  df2.columns = ["Comment_" + str(i + 1) for i in df2.columns]
  df2.index.name = "articleID"

  ##CONNECT THE COMMENTS TO THE INFORMATION ABOUT THE ARTICLE
  ## Throw away variables that seem less important, but we could add those later if we want.

  merged_dataframe = articles[['headline', 'keywords', 'sectionName', 'snippet', 'articleID']].merge(df2, on="articleID")

  merged_dataframe[:5]
  return merged_dataframe



In [12]:
connected_comments = connect_comments(feb2018comments, feb2018article)

connected_comments[:2]

,headline,keywords,sectionName,snippet,articleID,Comment_1,Comment_2,Comment_3,Comment_4,Comment_5,Comment_6,Comment_7,Comment_8,Comment_9,Comment_10,Comment_11,Comment_12,Comment_13,Comment_14,Comment_15,Comment_16,Comment_17,Comment_18,Comment_19,Comment_20,Comment_21,Comment_22,Comment_23,Comment_24,Comment_25,Comment_26,Comment_27,Comment_28,Comment_29,Comment_30,Comment_31,Comment_32,Comment_33,Comment_34,Comment_35,...,Comment_1745,Comment_1746,Comment_1747,Comment_1748,Comment_1749,Comment_1750,Comment_1751,Comment_1752,Comment_1753,Comment_1754,Comment_1755,Comment_1756,Comment_1757,Comment_1758,Comment_1759,Comment_1760,Comment_1761,Comment_1762,Comment_1763,Comment_1764,Comment_1765,Comment_1766,Comment_1767,Comment_1768,Comment_1769,Comment_1770,Comment_1771,Comment_1772,Comment_1773,Comment_1774,Comment_1775,Comment_1776,Comment_1777,Comment_1778,Comment_1779,Comment_1780,Comment_1781,Comment_1782,Comment_1783,Comment_1784
0,I Stand With the ‘She-Devils’,"['Women and Girls', ""Women's Rights"", 'United States Politics and Government', '#MeToo Movement'...",Unknown,"Progress on women’s rights was stalling, and then Donald Trump came along.",5a7258e410f40f00018bed7d,"[The snake-filled heads comment made me think of Medusa. <br/><br/>I hope he loses, so retrogra...","[She-devil reporting for duty!, False, 2, 25795675.0, 5a7258e410f40f00018bed7d]","[XX is the new mark of the devil., False, 1, 25792078.0, 5a7258e410f40f00018bed7d]","[""Courtland Sykes"" should be writing for The Onion, not running for office. , False, 6, 25791604...","[I happen to descend for a few of them, because we nasty women like to roam in PACs! Isabella, ...","[I don't care about the details of this troglodyte's oddities. Thing is, in much of America, the...","[As I have grown older, the more I have learned about sexism, the more I want it gone.<br/><br/>...",[“...a newfound freedom in the Age of Trump to say things that were previously beyond the pale.”...,"[""Many women, and some men, were horrified that a man who boasted of sexual assault could be ele...","[After reading about Sykes, I'm tempted to feel relief that McCaskill will have no problem holdi...","[I hate to say this about someone I love as much as I love you, Nicholas Kristof, but another re...",[Yes! Yet another reason to hold Mr Kristof in the highest esteem. The dignity of all humans i...,"[Courtland Sykes might do well to heed a cautionary tale from NJ where, last January, Atlantic C...","[I used to joke about being a ""Feminazi"". But I stopped. There's something retrograde about thes...","[""To be on the right side of history, we should all, men and women alike, stand on the side of t...",[While I see no value in Trump I see something more going on with the West.<br/>Here is what I s...,"[What is the source of the statistic that says in 1987, only half of Americans believed wife-bea...","[Good job, sir. I agree one hundred percent. The voice rising from fearful anxious men or their ...",[Some more encouraging news from flyover country (aka Missouri). We aren't all dinosaurs like Sy...,[Men are such delicate and fragile creatures; always needing reassurance of our strength and vir...,"[The comments regarding women, from some conservatives, are simply outdated. This is not an era ...","[Nicholas Kristof writes, “Why is a man writing this column? Frankly, because women’s rights are...",[Well it's quite clear Claire McCaskill will have no problem dispatching this anachronistic buff...,"[Thank you Mr. Kristoff! You are right, 'women's rights aren't just a woman's issues any more t...","[If I were Sykes I'd sign up for some cooking classes., False, 12, 25791841.0, 5a7258e410f40f000...",[Blessings on you from an fellow Oregonian! We admire your continued stands for women's issues....,"[A perfect column, Nick! Thank you<br/>very much.<br/>When Hillary Clinton lost the Electoral Co...",[The good news is -- it is much easier to fight against the tyranny of those you can identify. I...,"[

In [ ]:
# def clean(data):
#   cols = [x for x in data.columns if 'Comment' in x]
#   #for col in cols:
#   for col in cols:
#     row = data[col]
#     for y in range(0, len(row)):
#       data[col][y] = data[col][y][0].lower()
#     print(data[col][y])
#     # data[col] = data[col].apply(lambda x: x[0].apply(str))
#     #data.apply(lambda x: x.apply(str) if x.dtype == 'object' else x).col[0]
#     #.str.replace(".?!@#$%^></\\\\", "")
#   return data


# # def tokenize(data):
# #   import spacy
# #   nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
# #   from spacy.lang.en import English

# #   data = clean(data)
# #   tokenizer = English().Defaults.create_tokenizer(nlp)
# #   cols = [x for x in data.columns if 'Comment' in x]
# #   for col in cols:
# #     data[col] = tokenizer(data[col].apply(lambda y: tokenizer(y)))
# #   return data

# x = clean(connected_comments)
# print(x["Comment_1"])

In [ ]:
# def clean(data):
#   cols = [x for x in data.columns if 'Comment' in x]
#   for col in cols:
#     data[col] = data[col].astype(str).str.lower()
#     data[col] = data[col].astype(str).str.replace(".?!@#$%^></\\\\", "")
#   return data


# def tokenize(data):
#   import spacy
#   nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
#   from spacy.lang.en import English

#   data = clean(data)
#   tokenizer = English().Defaults.create_tokenizer(nlp)
#   cols = [x for x in data.columns if 'Comment' in x]
#   for col in cols:
#     data[col] = tokenizer(data[col][0].apply(lambda y: tokenizer(y.str)))
#   return data

# print(tokenize(connected_comments))

## Data Description

In [16]:
##Find out some statistics of entir NYtimes comments dataset
groupings = df_allcomments.groupby('articleID', as_index=False)
comments_count = groupings["commentBody"].count()

# print(comments_count)
print("MEAN = "+str(comments_count.mean()))
print("SD = "+str(comments_count.std()))
print("ALL ARTICLES = "+str(comments_count.count()))
print("ALL COMMENTS: = "+str(df_allcomments["commentID"].count()))
##MEAN = 230 comments per article
## SD = 403.4 comments    
##9450 articles
##And 2.176.364 comments

MEAN = commentBody    230.303069
dtype: float64
SD = commentBody    403.40704
dtype: float64
ALL ARTICLES = articleID      9450
commentBody    9450
dtype: int64
ALL COMMENTS: = 2176364


## Re-rank comments strategy 1: by user likes

In [17]:
import numpy as np
## See if we can re-rank comments by popularity (e.g. likes)
def get_liked(comments, articles, N=None):

  #Get a like rank
  comments['like_rank'] = comments.groupby('articleID')["recommendations"].rank(ascending=False, axis="rows")
  liked_comments_per_article = connect_comments(comments, articles, extra_comment_column='like_rank')

  #Change the df to a list of lists
  cols = [x for x in liked_comments_per_article.columns if 'Comment' in x]
  list_of_articles = [x.tolist() for x in liked_comments_per_article[cols].values]

  #Filter out non-existent comments
  list_of_articles = filter(None, list_of_articles)
  str_list = [list(filter(None, lst)) for lst in list_of_articles]

  #sort the list by likes (position 2 in the list of list)
  sorted_list = [sorted(x, key=lambda x: x[2], reverse=True) for x in str_list]

  #get the top N articles
  if N != None:
      sorted_list = [x[:N] for x in sorted_list]
  else:
      sorted_list = [x[:] for x in sorted_list]

  return sorted_list


# all_comments = get_liked(feb2018comments, feb2018article)

# #the most liked comments of the first 5 articles
# print("Top 3 for first 2 articles")
# print(top3_liked_comments[:1])
# print(len(top3_liked_comments[:1][0]))
# print("Top 5 for first 2 articles")
# print(top5_liked_comments[:1])
# print(len(top5_liked_comments[:1][0]))
# print("Top 10 for first 2 articles")
# print(top10_liked_comments[:1])
# print(len(top10_liked_comments[:1][0]))

# print("All comments for first 2 articles")
# print(all_comments[:1])


## Re-rank comments strategy 2: selected by NYTimes editors





In [18]:
## re-rank comments by whether or not recommended by NYTimes
pd.options.mode.chained_assignment = None 

def get_selected(comments, articles, N=None):
  selected_comments = comments.loc[comments["editorsSelection"] == True]

  selected_comments_per_article = connect_comments(selected_comments, articles)

  #Change the df to a list of lists
  cols = [x for x in selected_comments_per_article.columns if 'Comment' in x]
  list_of_articles = [x.tolist() for x in selected_comments_per_article[cols].values]

  # #Filter out non-existent comments
  #list_of_articles = filter(None, list_of_articles)
  str_list = [list(filter(None, lst)) for lst in list_of_articles]

  if N != None:
    str_list = [x[:N] for x in str_list]
  else:
    str_list = [x[:] for x in str_list]
  
  return str_list

# selected_comments = get_selected(feb2018comments, feb2018article)

# selected_comments[0]
# print(len(selected_comments[0]))
# ##11 comments are selected in article 1

# selected_comments[1]
# print(len(selected_comments[1]))
# ##10 comments are selected in article 2

# selected_comments[2]
# print(len(selected_comments[2]))
# ##9 comments are selected in article 2

# top3_selected_comments = get_selected(feb2018comments, feb2018article, N=3)
# top5_selected_comments = get_selected(feb2018comments, feb2018article, N=5)
# top10_selected_comments = get_selected(feb2018comments, feb2018article, N=10)

# #the most liked comments of the first 5 articles
# print("Top 3 for first 2 articles")
# print(top3_selected_comments[:1])
# print(len(top3_selected_comments[:1][0]))
# print("Top 5 for first 2 articles")
# print(top5_selected_comments[:1])
# print(len(top5_selected_comments[:1][0]))
# print("Top 10 for first 2 articles")
# print(top10_selected_comments[:1])
# print(len(top10_selected_comments[:1][0]))


In [26]:
##Find out some statistics of entir NYtimes comments dataset, only selected comments
# groupings = selected_comments.groupby('articleID', as_index=False)
import numpy as np
import scipy

selected_comments = get_selected(df_allcomments, df_allarticles)

selected_count = [len(x) for x in selected_comments]

# print(selected_count)
print("MEAN = "+str(np.mean(selected_count)))
print("MODE = "+str(max(set(selected_count), key=selected_count.count)))
print("SD = "+str(np.std(selected_count)))
print("ALL ARTICLES = "+str(len(selected_count)))
# print("ALL COMMENTS: = "+str(df_allcomments["commentID"].count()))
##MEAN = 13 selected comments per article (SD = 11)
##3646 selected in total


MEAN = 13.064781773263794
MODE = 1
SD = 11.337299962374988
ALL ARTICLES = 3643


## Re-rank comments strategy 3: diversity (max intra-list diversity)

In [ ]:
# !pip install transformers
# !pip install gensim

In [ ]:
# !brew install wget

# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
  ##For each comment;
  ##Change to a word vector;
  ##Then, maximize distance within word vector? Average word vector?
  #Using BERT?
  #over: author(s), tags, sections, and word embeddings. (As in Graus 2020?)

  #probably: re-using metric from Liu & Graus (2020), who use a more click-based and less normative idea of diversity.

  ##Using GoogleNews Vectors 

In [ ]:

def vectorize_comments(comments, article):
  connected_comments = connect_comments(comments, articles)

  return vectorized_comments


def intra_list_div(comments, articles, N=None):
  from sklearn.metrics.pairwise import cosine_similarity

  vectorized_comments = vectorize_comments(comments, articles)

  list_div = [1 - cosine_similarity(x, y) for x, y in zip(comments, comments[1:])] 
  
  diversity = list_div / (N * (N-1)/2) 

  return diversity



In [ ]:
# tokenize and pad every document 
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

# cols = [x for x in connected_comments.columns if 'Comment' in x]

# tokenizer=Tokenizer()

# tokenizer.fit_on_texts([connected_comments[x] for x in cols])
# tokenized_documents=tokenizer.texts_to_sequences([x[0] for x in connected_comments[cols]])
# tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
# vocab_size=len(tokenizer.word_index)+1
# print (tokenized_paded_documents[0])

In [ ]:
### GOOGLE VECTORS (from 2013, so older)
import gensim    
from gensim import models
from gensim.models import Word2Vec

#Downloaded from: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

##Open Vectors
import gzip

f = gzip.open('/content/Vectors.bin.gz', 'rb')
file_content = f.read()
f.close()


In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/Vectors.bin.gz', binary=True)
dog = model['dog']
print(dog.shape)

(300,)


In [ ]:
##Using BERT
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

encoded_input = tokenizer("Hello, I'm a single sentence!")
print(encoded_input)


{'input_ids': [101, 8667, 117, 146, 112, 182, 170, 1423, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


##Theory-based diversity metrics

###Activation metric

In [37]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 8.5MB/s 


In [27]:
def polarity(comment_list, articles=None):
  from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
  from statistics import mean

  analyser = SentimentIntensityAnalyzer()
  #comments_nr = []

  #For every article
  for y in range(0, len(comment_list)):
    #print("NEW ARTICLE")
    article_list = []
    text_list = []
    #print("nr of comments:")
    #comments_nr.append(comment_list[y])
    #print(len(comment_list[y]))
    for x in comment_list[y]:
      #score of every comment
      article_list.append(analyser.polarity_scores(str(x[0]))['compound'])
      text_list.append(str(x[0]))
    # #Mean of every article
    #print("EVERY COMMENT SCORE IN RECOMMENDATION LIST")
    #print(article_list)
    #print(text_list)
    comment_list[y] = abs(mean(article_list))
    
    #print("MEAN ABSOLUTE SCORE OF RECOMMENDATION LIST")
    #print(abs(mean(article_list)))

  if articles != None:
    for y in range(0, len(articles)):
      #print("NEW ARTICLE")
      article_list = []
      text_list = []
      #print("nr of comments:")
      #comments_nr.append(comment_list[y])
      #print(len(comment_list[y]))
      for x in comment_list[y]:
        #score of every comment
        article_list.append(analyser.polarity_scores(str(x[0]))['compound'])
        text_list.append(str(x[0]))
      #print("EVERY COMMENT SCORE IN RECOMMENDATION LIST")
      #print(article_list)
      #print(text_list)
      comment_list[y] = abs(mean(article_list))
     # print("MEAN ABSOLUTE SCORE OF RECOMMENDATION LIST")
     # print(abs(mean(article_list)))

  #print("MEAN NR COMMENTS")
  #print(mean(comments_nr))
  #print(std(comments_nr))
  return comment_list

def activation_metric(possible_comments, recommended_comments, articles=None):
  from statistics import mean

  # print("POSSIBLE")
  possible_score = polarity(possible_comments, articles=articles)

  # print("RECOMMENDED")
  recommended_score = polarity(recommended_comments, articles=articles)
  
  #print("mean absolute sentiment score of ALL recommendation lists")
  score = (abs(mean(possible_score)) - abs(mean(recommended_score)))/2 
  #print("score = absolute_mean_possible_comments - absolute_mean_recommended_comments")
  #print(str(score)+" = "+str(abs(mean(possible_score))) +" - "+str(abs(mean(recommended_score)))+"/2" )
  # print(str(score)+"="+str(abs(possible_score))+(" - ")+str(mean(recommended_score))))
  return score

####Data: Liked comments

In [28]:
top3_liked_comments = get_liked(feb2017comments, feb2017article, N=3)

In [29]:
top5_liked_comments = get_liked(feb2017comments, feb2017article, N=5)

In [30]:
top10_liked_comments = get_liked(feb2017comments, feb2017article, N=10)

In [31]:
all_comments = get_liked(feb2017comments, feb2017article)

####Data: Selected Comments

In [32]:
top3_selected_comments = get_selected(feb2017comments, feb2017article, N=3)

In [33]:
top5_selected_comments = get_selected(feb2017comments, feb2017article, N=5)

In [34]:
top10_selected_comments = get_selected(feb2017comments, feb2017article, N=10)

In [35]:
selected_comments = get_selected(feb2017comments, feb2017article)

####Activation Test: Liked

In [ ]:
##Top3 Liked
print(activation_metric(all_comments, top3_liked_comments))

In [ ]:
##Top5 Liked
print(activation_metric(all_comments, top5_liked_comments))

-0.052061708146710045


In [ ]:
##Top10 Liked
print(activation_metric(all_comments, top10_liked_comments))

-0.034534571118640806


#### Activation Test: Selected

In [ ]:
##Top3 selected
print(activation_metric(all_comments, top3_selected_comments))

-0.06716668272298124


In [ ]:
##Top5 selected
print(activation_metric(all_comments, top5_selected_comments))

-0.0381701083789579


In [ ]:
##Top10 selected
print(activation_metric(all_comments, top10_selected_comments))

-0.020918482031139857


In [ ]:
##SELECTED VS NON
print(activation_metric(all_comments, selected_comments))

-0.012693080528571876


###Test_print

In [ ]:
# #top3 recommended for article 1 (recommended)
# top3_liked_comments = get_liked(feb2018comments, feb2018article, N=3)

# print(top3_liked_comments[0:2]) 

# print(len(top3_liked_comments[0]))

# # all for article 1 (possible)
# all_comments = get_liked(feb2018comments, feb2018article)

# print(all_comments[0:2])

# print(len(all_comments[0:2]))


[[["For all of my 43-years I've heard men try to tell women, and sometimes specifically me, how be: not too shrill, not too ambitious, not too muscular, not too foul-mouthed, well-dressed, HWP, good at making dinner by 6pm, and take care not to be a know-it-all (read: question your male colleagues).<br/><br/>Sykes is the kind of guy that is all-too-willing to continue that trend, even though we are now living in an age when it is perfectly OK to question this kind of patriarchical condescension, and to laugh at it, as women. Men like him were very confusing when I was a young woman, because no one was writing columns like this. No one was questioning the fact that, maybe, women and girls didn't want to be put neatly into categories. <br/><br/>I am so grateful to be living in a time when I can openly thumb my nose, and worse, at guys like him. And I'm grateful for men like Nicholas Kristof who will back us up! The future looks bright for everyone, regardless of gender. ", True, 543, 257

In [ ]:
# rec = top3_liked_comments[:2]
# poss = all_comments[:2]

# print(rec[0])
# print(poss[0])

# for y in range(0, len(rec)):
#   article_list = []
#   for x in  rec[y]:
#     # print("COMMENT")
#     article_list.append(analyser.polarity_scores(str(x))['compound'])
#   rec[y] = article_list

# # print(rec)


# for y in range(0, len(poss)):
#   article_list = []
#   for x in  poss[y]:
#     # print("COMMENT")
#     article_list.append(analyser.polarity_scores(str(x))['compound'])
#   poss[y] = article_list

# # print(poss)

# print(rec[0])
# print(poss[0])



In [ ]:
#top3_liked_comments[:5]

list_of_articles = [x[0] for y in top3_liked_comments for x in y]

# [x[0] for x in top3_liked_comments]

for i in list_of_articles:
  print(i)
  print("xxxx")


Streaming output truncated to the last 5000 lines.
I was at a talk Judy Chicago gave at the Brooklyn Museum. She kicked out a young woman out of the auditorium who had a small child making very minimal noise in the back row. I met the woman outside. She was crying and told me she loved Chicago and could not get a babysitter.<br/><br/>It is also my understanding that Chicago did not pay nor aknowledge any of the women who worked on the dinner party. <br/><br/>Cruelly toward women with kids. Free women's labor. That's not what I want from my feminism. (BTW, I'm a woman, Jim isn't my name).
xxxx
Two of the three genes for light skin spread to Europe from the Middle East after the advent of agriculture. That they arose in the first place and then spread rapidly points to agriculture as the likely cause.<br/><br/>Pre-agricultural peoples ate a diet high in meat and fish, and so rich in Vitamin D. The agricultural diet contained much less Vitamin D than the hunter-gatherer diet, meaning more

###Representation metric

In [ ]:
def representation_metric(possible_comments, recommended_comments):
  #Kullblack-Leibler Divergence, different opinion categories
  # different opinion categories --> metadata?
  return score

###Protected Voices Metric

In [ ]:
def count_protected(comments)
  return count

def count_unprotected(comments)
  return count

def voices_metric(possible_comments, recommended_comments):
  protected_group_recommended = count_protected(recommended_comments)
  protected_group_possible = count_protected(possible_comments)

  unprotected_group_recommended = count_unprotected(recommended_comments)
  unprotected_group_possible = count_unprotected(possible_comments)

  score = (protected_group_recommended/protected_group_possible) / (unprotected_group_recommended/unprocted_group_recommended)
  return score

#Experiments

#Evaluation of the test set

# **SFU Dataset**


## Loading the data

In [ ]:
#Importing the zip
zip_ref_sfu = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/SFU-Opinion_and_Comments_Corpus.zip", "r")
zip_ref_sfu.extractall()
zip_ref_sfu.close()

In [ ]:
import pandas as pd

# Raw data
sfu_articles = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gnm_articles.csv")
sfu_comments = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gnm_comments.csv")
sfu_threads = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gnm_comment_threads.csv")

threads.head(5)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,9,10,11,13,14,16,17,18,23,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,article_id,comment_counter,comment_author,timestamp,post_time,comment_text,TotalVotes,posVotes,negVotes,vote,reactions,replies,comment_id,parentID,threadID,streamId,edited,isModerator,highlightGroups,moderatorEdit,descendantsCount,threadTimestamp,flagCount,sender_isSelf,sender_loginProvider,data_type,is_empty,status
0,535381,source1_535381_0,Muhammad Shoaib,1.461225e+12,NaN,Indeed it is the best police force in the worl...,0.0,0.0,0.0,none,NaN,NaN,bca183ec85d04ba9abe7df742b05baef,NaN,NaN,535381.0,False,False,[],NaN,NaN,1.461225e+12,0.0,False,Facebook,comment,N,published
1,543638,source1_543638_0,Franklin Galvin1,1.454088e+12,NaN,Patients need to express their discontent with...,0.0,0.0,0.0,none,NaN,NaN,3f349e7b7c6b4a73bbe240b86ca81ec1,NaN,NaN,543638.0,False,False,[],NaN,NaN,1.454088e+12,0.0,False,Site,comment,N,published
2,544184,source1_544184_0,openwater2010,1.404260e+12,NaN,The Opinion Gap on SPAM seems to match the opi...,1.0,1.0,0.0,none,NaN,NaN,672e3742723545b4bfa745594d1673e1,NaN,NaN,544184.0,False,False,[],NaN,NaN,1.404260e+12,0.0,False,Site,comment,N,published
3,544562,source1_544562_0,Wild Bill 2,1.388010e+12,NaN,Prince Rupert route would avoid having 220 sup...,0.0,NaN,NaN,none,NaN,NaN,73767f8e40c941d1bfdba7b8e8f86b03,NaN,NaN,544562.0,False,False,[],NaN,NaN,1.388010e+12,0.0,False,Site,comment,N,published
4,544912,source1_544912_0,SusanK.,1.378332e+12,NaN,"If I understand your commentary, you are targe...",-4.0,NaN,4.0,none,NaN,NaN,6b6933f1beff41b3934ca71f7aab3ab5,NaN,NaN,544912.0,False,False,[],NaN,NaN,1.378332e+12,0.0,False,Site,comment,N,published


In [ ]:
sfu_constr = pd.read_csv("/content/SOCC/annotated/constructiveness/SFU_constructiveness_toxicity_corpus.csv")
sfu_constr.head()

,article_id,comment_counter,title,globe_url,url,comment_text,is_constructive,is_constructive:confidence,toxicity_level,toxicity_level:confidence,did_you_read_the_article,did_you_read_the_article:confidence,annotator_comments,expert_is_constructive,expert_toxicity_level,expert_comments
0,23462276,source1_23462276_4,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,"While technology does march on, sometimes it t...",yes,1.0000,1,1,1,1.0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
1,23462276,source1_23462276_10,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,Everyone is still missing the point of what th...,yes,1.0000,1\n2,0.9111\n0.0889,1,1.0,\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
2,23462276,source1_23462276_11_1,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,You've never used an Apple product have you?,no,1.0000,1\n2,0.6308\n0.2937,1,1.0,\n\n\n\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
3,23462276,source1_23462276_25,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,You may be using a blackberry. I'm still using...,no,1.0000,1\n2,0.7346\n0.2023,1,1.0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
4,26373964,source1_26373964_63,Don't be fooled by the (surprise!) budget surplus,http://www.theglobeandmail.com/opinion/dont-be...,http://www.sfu.ca/content/dam/sfu/discourse-la...,"Of course we all know, unfortunately, there is...",no,0.5963,2\n1,0.4429\n0.3066,1,1.0,\n\n\n\n\n\n,NaN,NaN,NaN


In [ ]:
#Getting the constructiveness annotations
zip_ref_sfu_app = zipfile.ZipFile("/content/SOCC/annotated/Appraisal/Appraisal_annotations.zip", "r")
zip_ref_sfu_app.extractall()
zip_ref_sfu_app.close()

## Data description

## What data do we have
- 10.339 opinion articles from The Globe and Mail
- 663.173 comments linked to those articles
- 303.665 comment threads in theory; in practice it's 773716 which makes absolutly no sense at all but I checked the raw file and it does indeed have that many lines
- 1043 annotated comments in terms of their constructiveness
- Raw metrics for **comments** (such as pos/neg votes, replies and reactions, but also thread id's and positions) and **articles** (text, headlines, and number of (top-level) comments per article)
- Dictionaries with annotations for appraisal and negation (not quite sure yet how to transform them in a way that I really understand what is in there and how it is organised)
### What would we need to do with the data
- Article text needs to be cleaned, although it seems the only issues are related to formatting: < /p>" & < p>
- **Not use** the threads data because it clearly seems untrustworthy
- Link comments to articles (either with the article id or the thread variable)
- Unpack the dictionaries??

In [ ]:
#How much raw data is there
print("Articles:",len(sfu_articles),
      "Comments:", len(sfu_comments),
      "Threads:", len(sfu_threads),
      "Constructiveness:", len(sfu_constr))

Articles: 10339 Comments: 663173 Threads: 773716 Constructiveness: 1043


In [ ]:
#What is in the article data?
sfu_articles.columns

Index(['article_id', 'title', 'article_url', 'author', 'published_date',
       'ncomments', 'ntop_level_comments', 'article_text'],
      dtype='object')

In [ ]:
#What is in the columns data?
sfu_comments.columns

Index(['article_id', 'comment_counter', 'comment_author', 'timestamp',
       'post_time', 'comment_text', 'TotalVotes', 'posVotes', 'negVotes',
       'vote', 'reactions', 'replies', 'comment_id', 'parentID', 'threadID',
       'streamId', 'edited', 'isModerator', 'highlightGroups', 'moderatorEdit',
       'descendantsCount', 'threadTimestamp', 'flagCount', 'sender_isSelf',
       'sender_loginProvider', 'data_type', 'is_empty', 'status'],
      dtype='object')

In [ ]:
#What is in the constructiveness data?
sfu_constr.columns

Index(['article_id', 'comment_counter', 'title', 'globe_url', 'url',
       'comment_text', 'is_constructive', 'is_constructive:confidence',
       'toxicity_level', 'toxicity_level:confidence',
       'did_you_read_the_article', 'did_you_read_the_article:confidence',
       'annotator_comments', 'expert_is_constructive', 'expert_toxicity_level',
       'expert_comments'],
      dtype='object')

In [ ]:
#What does the text look like?
articles["article_text"][20]

"<p>It's good to be a firefighter, especially if you live in a small town. You're one of the best-paid guys around. You get lots and lots of time off to go fishing or hunting or run your own business on the side. And your union always gets its way.</p><p>There's just one problem. The people you work for can't afford you anymore.</p><p>'We're spending 25 per cent of our operating budget on firefighters,' Ted Luciani, the mayor of Thorold, Ont., told me the other day. 'It's not sustainable. We've hit the wall.'</p><p>Thorold is a middle-class town of 18,500 in southwestern Ontario. It has 15 significant structure-related fires a year, the mayor's office says. But although its 18 firefighters don't have much to do, they make big-city money. An arbitrator recently awarded them a retroactive 9.2 per cent raise that bumped their pay to $92,119. Similar settlements have been awarded throughout the province. Most of the people who make these towns' Sunshine List (because their compensation is 

In [ ]:
#appraisal dictionary
import json

file = "exportedproject5207298023855609783.json"
#with open(file) as appraisal:
#    dict_appraisal = json.load(appraisal)

#print(pd.json_normalize(dict_appraisal))


        name description  ... disableExport  script_direction
0  Appraisal        None  ...         False               LTR

[1 rows x 12 columns]


In [ ]:
df = pd.read_json(file,orient='index')
df

,0
name,Appraisal
description,None
mode,ANNOTATION
version,1
source_documents,"[{'name': 'aboriginal_1.txt', 'format': 'text'..."
annotation_documents,"[{'name': 'aboriginal_1.txt', 'user': 'Luca', ..."
project_permissions,"[{'level': 'USER', 'user': 'Erin'}, {'level': ..."
tag_sets,"[{'name': 'Appraisal', 'typeUiName': None, 'de..."
layers,"[{'name': 'webanno.custom.Attitude__', 'featur..."
mira_templates,[]
